In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns

import xarray as xr

import kinnd

sns.set_style("darkgrid")

In [ ]:
listen_fpath = kinnd.paths.get_listen_path()
ds = xr.open_zarr(str( listen_fpath / "data" / "derivatives" / "xarray" / "phonemes_proc-MneBidsPipeline_evoked.zarr"))
ds

In [ ]:
dat = ds.sel(channel=["Cz", "E7", "E106"], condition="tone/deviant").mean(["channel", "time"]).values()

plt.scatter(x=dat, y=list(ds.var()))

In [ ]:
from scipy.stats import zscore

df_outlier = ds.sel(channel=["Cz", "E7", "E106"], condition="tone/deviant").mean(["channel", "time"]).to_pandas()
zscores = df_outlier.to_frame().rename(columns={0: "z"}).apply(zscore)
zscores.loc[zscores["z"] > 3]

In [ ]:
df = ds.sel(channel=["Cz", "E7", "E106"]).mean("channel").to_dataframe()

df = df.reset_index().melt(
    id_vars=["condition", "time"],
    var_name="subject",
    value_vars=[col for col in df.columns if col.startswith("sub")],
    value_name="voltage",
)

# drop outliers that were defined in cell above
idx_to_drop = df.loc[df["subject"] == "sub-2041"].index
df = df.drop(idx_to_drop)

# Merged Group demographic info
group_fpath = listen_fpath / "data" / "listen_demographics.csv"
group_df = pd.read_csv(group_fpath)
df = df.merge(group_df, on="subject")
df

In [ ]:
width = 7 # inches
height = 5 # inches

fig, ax = plt.subplots(figsize=(width, height), nrows=3, ncols=2, constrained_layout=True)

g = sns.FacetGrid(data=df, col="Group", col_wrap=2)
g.map_dataframe(sns.lineplot, x="time", y="voltage", hue="condition")

g.add_legend()

In [ ]:
g.savefig("./phonemes_evoked.png", dpi=300)

In [ ]:
sns.lineplot(data=df, x="time", y="voltage", hue="condition")